# Habituation to turbulent odor backgrounds
Look at a 6-odor case. Compare BioPCA and IBCM models for habituation and new odor detection. 

# Correlating turbulent background processes

We first simulate independent concentration variables $\nu(t)$ and consider their zero-meaned version, $\tilde{\nu}(t) = \nu(t) - \langle \nu \rangle$. These variables have mean zero, a diagonal covariance matrix $\sigma^2 \mathbb{1}$, and third moment $m_3$ (while mixed third-order moments of different $\tilde{\nu}_\mu$ are zero due to their zero average). 

Then, we can transform these variables to have some desired covariance matrix $C$ with Cholesky decomposition $\sigma^2 RR^T = \Sigma$ (i.e. $R$ is the Cholesky decomposition of $\Sigma / \sigma^2$ and thus has scale $1/\sigma$). Indeed, we take concentrations $c = R \tilde{\nu} + \langle \nu \rangle$. These have the same mean as the original variables, $\langle c \rangle = \langle \nu \rangle$, and the desired covariance matrix, 

$$ C = \langle (c - \langle c \rangle) (c - \langle c \rangle)^T \rangle = \langle R \tilde{\nu} \tilde{\nu}^T R^T \rangle = \sigma^2 R R^T = \Sigma $$

The third moments are altered by this transformation, but they are not generally zero, so we can still numerically get the IBCM model to converge. 


Let's use as a target the covariance matrix

$$ \Sigma = \sigma^2 \begin{pmatrix}
    1 & \rho & \ldots & \rho \\
    \rho & 1 & \ldots & \rho \\
    \ldots & \ldots & \ldots & \ldots \\
    \rho & \rho & \ldots & 1
\end{pmatrix} $$

where $-1 < \rho < 1$ is the Pearson correlation coefficient. Then $R$ is the Cholesky decomposition of the matrix on the right. 

### Asymmetry between odors?
If we instead keep turbulent odors independent, but give them different fluctuation magnitudes, for instance forcing the following covariance matrix with the same Cholesky decomposition trick:
$$
    \Sigma = \begin{pmatrix}
    \frac12 \sigma^2 (1 + u_0) & 0 & \ldots & 0 \\
    0 & \frac12 \sigma^2 (1 + u_1) & \ldots & 0 \\
    \ldots & \ldots & \ldots & \ldots \\
    0 & 0 & \ldots & \frac12 \sigma^2 (1 + u_n)
\end{pmatrix} $$
with $u_i \sim U(0, 1)$ uniform random samples, then the IBCM network still gets neurons specific to each odor. So asymmetry in magnitude between background odors is not a major challenge for this model. It is really the difference in temporal statistics between odors introduced by the Cholesky approach to correlate them that is the issue: since odor $n$ is a mixture of all $n$ previous underlying processes, it gets a lot intra-whiff fluctuations from all other odors' fluctuations. So each odor's temporal fluctuations have different scales, and fixed points aligned with some odors thus become more stable than others, with the network potentially missing some background components for that reason. 

## Imports

In [ ]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from time import perf_counter
import os, json, sys
if ".." not in sys.path:
    sys.path.insert(1, "..")
    
from sklearn.decomposition import FastICA

from modelfcts.ibcm import (
    integrate_inhib_ibcm_network_options,
    ibcm_respond_new_odors,
    compute_mbars_cgammas_cbargammas,
    ibcm_respond_new_odors
)
from modelfcts.ibcm_analytics import (
    fixedpoint_thirdmoment_exact, 
    ibcm_fixedpoint_w_thirdmoment, 
    ibcm_all_largest_eigenvalues
)
from modelfcts.biopca import (
    integrate_inhib_biopca_network_skip,
    build_lambda_matrix,
    biopca_respond_new_odors
)
from modelfcts.average_sub import (
    integrate_inhib_average_sub_skip, 
    average_sub_respond_new_odors
)
from modelfcts.ideal import (
    find_projector, 
    find_parallel_component, 
    ideal_linear_inhibitor, 
    compute_ideal_factor
)
from modelfcts.checktools import (
    analyze_pca_learning, 
    check_conc_samples_powerlaw_exp1
)
from modelfcts.backgrounds import (
    update_powerlaw_mixed_concs,
    logof10, 
    sample_ss_conc_powerlaw,
    sample_ss_mixed_concs_powerlaw,
    generate_odorant
)
from modelfcts.tagging import (
    project_neural_tag, 
    create_sparse_proj_mat, 
    SparseNDArray, 
)
from utils.statistics import seed_from_gen
from modelfcts.distribs import (
    truncexp1_average,
    powerlaw_cutoff_inverse_transform
)
from utils.smoothing_function import (
    moving_average, 
    moving_var
)
from simulfcts.plotting import (
    plot_cbars_gamma_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats
from utils.metrics import jaccard

In [ ]:
from utils.metrics import l2_norm, l1_norm, linf_norm, cosine_dist

def distance_panel_target(mixes, target):
    """ Compute a panel of distances between the pure (target) new odor and mixtures 
    (which can be without inhibition, with average inhibition, IBCM inhibition, etc.). 
    
    Four distances included, in order: l2, l1, linf, cosine_dist
    
    Args:
        mixes (np.ndarray): mixtures of odors to compute distance from target, 
            the last axis should have the size of target, 
            while other axes are arbitrary.  
        target (np.1darray): target odor vector, same length as
            last axis of mixes. 
    Returns:
        dist_panel (np.ndarray): shape of pure, except the last axis, 
            which has length 4 (for the number of distances computed). 
    """
    # Make axis 0 the axis indexing distance metrics, to begin with
    # And move it to the last axis before returning
    dist_array = np.zeros([4] + list(mixes.shape[:-1]))
    # No need to add axes to target vector; if it is 1d, it is broadcasted
    # along the last axis of mixes, which indexes elements of each vector. 
    dist_array[0] = l2_norm(target - mixes)
    dist_array[1] = l1_norm(target - mixes)
    dist_array[2] = linf_norm(target - mixes)
    dist_array[3] = cosine_dist(target, mixes)
    
    return np.moveaxis(dist_array, 0, -1)

### Aesthetic parameters

In [ ]:
#plt.style.use(['dark_background'])
plt.rcParams["figure.figsize"] = (4.5, 3.0)

In [ ]:
do_save_outputs = False

models = ["ibcm", "biopca", "avgsub", "ideal", "orthogonal", "none"]
model_nice_names = {
    "ibcm": "IBCM",
    "biopca": "BioPCA",
    "avgsub": "Average",
    "ideal": "Ideal",
    "optimal": "Optimal",
    "orthogonal": "Orthogonal",
    "none": "None"
}
model_colors = {
    "ibcm": "xkcd:turquoise",
    "biopca": "xkcd:orangey brown",
    "avgsub": "xkcd:navy blue",
    "optimal": "xkcd:powder blue",
    "ideal": "xkcd:light green",
    "orthogonal": "xkcd:pale rose",
    "none": "grey"
}

### Initialization

In [ ]:
# Initialize common simulation parameters
n_dimensions = 50  # Half the real number for faster simulations
n_components = 4  # Number of background odors

inhib_rates = [5e-5, 1e-5]  # alpha, beta  [0.00025, 0.00005]

# Simulation duration
duration = 360000.0
deltat = 1.0
n_chunks = 1
skp = 10 * int(1.0 / deltat)

# Common model options
activ_function = "identity"  # "ReLU"

# Background process
update_fct = update_powerlaw_mixed_concs

# Choose randomly generated background vectors
# This seed gave nicely spread out odors easier to learn 0xe329714605b83365e67b44ed7e001ec
rgen_meta = np.random.default_rng(seed=0xe629712605b83365e67b24ed7e001ec)
back_components = np.zeros([n_components, n_dimensions])
for i in range(n_components):
    back_components[i] = generate_odorant(n_dimensions, rgen_meta, lambda_in=0.1)
back_components = back_components / l2_norm(back_components).reshape(-1, 1)

# Seed for background simulation, to make sure all models are the same
simul_seed = seed_from_gen(rgen_meta)

# Turbulent background parameters: same rates and constants for all odors
back_params = [
    np.asarray([1.0] * n_components),        # whiff_tmins
    np.asarray([500.] * n_components),       # whiff_tmaxs
    np.asarray([1.0] * n_components),        # blank_tmins
    np.asarray([800.0] * n_components),      # blank_tmaxs
    np.asarray([0.6] * n_components),        # c0s
    np.asarray([0.5] * n_components),        # alphas
]

# Choose desired Pearson correlation between odors
# Up to 0.5 there is some convergence, beyond, issue. 
# Note that for rho = 1, there would effectively be only one odor. 
correl_rho = -0.6
all_correl = False

# Compute mean of independent underlying variables, 
# to determine the mean and target covariance of mixed variables
#indep_conc_samples = sample_ss_conc_powerlaw(*back_params, size=int(1e5))
#mean_conc_empirical = np.mean(indep_conc_samples)
#vari_conc = np.var(indep_conc_samples)
tblo, tbhi, twlo, twhi = back_params[2], back_params[3], back_params[0], back_params[1]
whiffprob = np.mean(1.0 / (1.0 + np.sqrt(tblo*tbhi/twlo/twhi)))
avg_whiff_conc = np.mean(truncexp1_average(*back_params[4:6]))
mean_conc = whiffprob * avg_whiff_conc  # average time in whiffs vs blanks * average whiff conc
print("Analytical mean conc:", mean_conc)
#print("Numerical mean conc:", mean_conc_empirical)

# Target covariance matrix (scaled by variance of underlying independent variables)
if all_correl:
    target_covmat_scaled = np.full((n_components, n_components), correl_rho)
    target_covmat_scaled[np.diag_indices(n_components)] = 1.0
    target_cholesky = np.linalg.cholesky(target_covmat_scaled)
    # Independent but unequal amplitude for odors: IBCM still finds all components
    # so asymmetry in scale (within reasonable range) is not an issue
    # Temporal statistics differences between odors coming from Cholesky mixing
    # of turbulent processes are a tougher challenge for IBCM. 
    #target_covmat_scaled = np.zeros((n_components, n_components))
    #target_covmat_scaled[np.diag_indices(n_components)] = 0.5 * (1.0 + rgen_meta.random(size=n_components))
    #target_cholesky = np.linalg.cholesky(target_covmat_scaled)
# Other option where we correlate only a pair of odors
else:
    target_covmat_scaled = np.zeros((n_components, n_components))
    target_covmat_scaled[[-1, -2], [-2, -1]] = correl_rho
    target_covmat_scaled[np.diag_indices(n_components)] = 1.0
    if abs(correl_rho) < 1.0:
        target_cholesky = np.linalg.cholesky(target_covmat_scaled)
    else:
        target_cholesky = np.zeros((n_components, n_components))
        target_cholesky[np.diag_indices(n_components)] = 1.0
        target_cholesky[-1, -1] = 0.0
        target_cholesky[-1, -2] = correl_rho  # Replace odor 1 by odor 0 for rho = +-1
print(target_cholesky)
print(target_covmat_scaled)
# Add mean conc and Cholesky mixing matrix to parameters
back_params.append(mean_conc)
back_params.append(target_cholesky)
# Then add background odor vectors last to that list
back_params.append(back_components)

# Initial values of background process variables (underlying independent (t, c))
init_concs_ind = sample_ss_conc_powerlaw(*back_params[:-3], size=1, rgen=rgen_meta)
init_times = powerlaw_cutoff_inverse_transform(
                rgen_meta.random(size=n_components), *back_params[2:4])
tc_init = np.stack([init_times, init_concs_ind.squeeze()], axis=1)

# Initial background vector 
init_concs_mix = target_cholesky.dot(tc_init[:, 1] - mean_conc) + mean_conc
init_bkvec = init_concs_mix.dot(back_components)
# nus are first in the list of initial background params
init_back_list = [tc_init, init_bkvec]

### Pairwise similarity between background odors
Determines how well-posed the PCA is and how easy it is for the IBCM model to disentangle odors


## IBCM habituation
### IBCM simulation

In [ ]:
# IBCM model parameters
n_i_ibcm = 24  # Number of inhibitory neurons for IBCM case

# Model rates
learnrate_ibcm = 0.00125 #5e-5
tau_avg_ibcm = 1200  # 2000
coupling_eta_ibcm = 0.6/n_i_ibcm
ssat_ibcm = 50.0
k_c2bar_avg = 0.1
decay_relative_ibcm = 0.005
lambd_ibcm = 1.0
ibcm_rates = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm 
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "tanh", 
    "variant": "law", 
    "decay": True
}

# Initial synaptic weights: small positive noise
init_synapses_ibcm = 0.5*rgen_meta.standard_normal(size=[n_i_ibcm, n_dimensions])*lambd_ibcm
#init_synapses_ibcm = (0.3 * back_components[rgen_meta.choice(n_components, size=n_i_ibcm), :]
#                      + 0.1*rgen_meta.standard_normal(size=[n_i_ibcm, n_dimensions]))* lambd_ibcm

In [ ]:
def recompute_theta_series(cbser, tau, dt):
    theta = np.zeros([cbser.shape[0], cbser.shape[1]])
    theta[0] = cbser[0]**2
    for i in range(cbser.shape[0]-1):
        theta[i+1] = theta[i] + dt/tau*(cbser[i]*cbser[i] - theta[i])
    return theta

In [ ]:
# Run the IBCM simulations
# Perform successive shorter runs/restarts for memory efficiency
tser_ibcm = []
nuser_ibcm = []
bkvecser_ibcm = []
mser_ibcm = []
cbarser_ibcm = []
wser_ibcm = []
yser_ibcm = []
thetaser_ibcm = []
if n_chunks > 1:
    seed_spawns = np.random.SeedSequence(simul_seed).spawn(10)
else:
    seed_spawns = [simul_seed]
for i in range(n_chunks):
    tstart = perf_counter()
    if i == 0:
        init_vari = init_synapses_ibcm
        init_back = init_back_list
    else:
        init_vari = [mser_ibcm[i-1][-1], thetaser_ibcm[i-1][-1], wser_ibcm[i-1][-1]]
        init_back = [nuser_ibcm[i-1][-1], bkvecser_ibcm[i-1][-1]]
    sim_results = integrate_inhib_ibcm_network_options(
                init_vari, update_fct, init_back, 
                ibcm_rates, inhib_rates, back_params, duration/n_chunks, 
                deltat, seed=seed_spawns[i], noisetype="uniform",  
                skp=skp, **ibcm_options
    )
    tser_ibcm.append(sim_results[0] + i/n_chunks*duration)
    nuser_ibcm.append(sim_results[1])
    bkvecser_ibcm.append(sim_results[2])
    mser_ibcm.append(sim_results[3]) 
    cbarser_ibcm.append(sim_results[4]) 
    thetaser_ibcm.append(sim_results[5])
    wser_ibcm.append(sim_results[6])
    yser_ibcm.append(sim_results[7])
    tend = perf_counter()
    print("Finished chunk", i, "in {:.2f} s".format(tend - tstart))

# Concatenate
tser_ibcm = np.concatenate(tser_ibcm, axis=0)
nuser_ibcm = np.concatenate(nuser_ibcm)
bkvecser_ibcm = np.concatenate(bkvecser_ibcm)
mser_ibcm = np.concatenate(mser_ibcm)
cbarser_ibcm = np.concatenate(cbarser_ibcm)
thetaser_ibcm = np.concatenate(thetaser_ibcm)
wser_ibcm = np.concatenate(wser_ibcm)
yser_ibcm = np.concatenate(yser_ibcm)

### Check background process

In [ ]:
# Mixed concentrations time series
conc_ser = np.einsum("ij,kj->ki", target_cholesky, nuser_ibcm[:, :, 1] - mean_conc) + mean_conc
print(conc_ser.shape)
fig, ax = plt.subplots()
tslice = slice(0, 500)
for i in range(n_components):
    ax.plot(tser_ibcm[tslice]/1000, conc_ser[tslice, i], lw=0.8, label="Odor {}".format(i))
ax.set(xlabel="Time (x1000 steps)", ylabel=r"Mixed odor concentrations")
ax.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))
plt.show()
plt.close()

In [ ]:
# Mixed concentrations time series
tslice = slice(0, 50000, 100)
n_cols = 6
n_plots = n_dimensions // 2
n_rows = n_plots // n_cols + min(1, n_plots % n_cols)
fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True)
fig.set_size_inches(n_cols*1.75, n_rows*1.75)
for i in range(n_plots):
    ax = axes.flat[i]
    ax.scatter(bkvecser_ibcm[tslice, 2*i+1], bkvecser_ibcm[tslice, 2*i], 
               s=9, alpha=0.5, color="k")
    for j in range(n_components):
        ax.plot(*zip([0.0, 0.0], 3.0*back_components[j, 2*i:2*i+2:][::-1]), lw=2.0)
    ax.set(xlabel="OSN {}".format(2*i+2), ylabel="OSN {}".format(2*i+1))
for i in range(n_plots, n_rows*n_cols):
    axes.flat[i].set_axis_off()
fig.tight_layout()
plt.show()
plt.close()

#### Remark

If the background has very strong correlation between odors, it may not be effectively $N_\mathrm{B}$-dimensional anymore, so the representation in terms of the dot products with original odors, $h_{\gamma}$, may not be appropriate. There could be a new basis on which the neurons pick up; some decomposition of the mixture $\sum_\gamma c_\gamma \mathbf{y}_\gamma$. Try the SVD, compute the dot products with the first two singular vectors. 

In [ ]:
from matplotlib.colors import Normalize, SymLogNorm

### IBCM habituation analysis

In [ ]:
# Calculate cgammas_bar and mbars
transient = int(5/6*duration / deltat) // skp
basis = back_components #recovered_components_ordered  # back_components
if correl_rho == 1.0:  # effectively N-1 vectors only
    basis = np.concatenate([back_components[:2], 
        np.sum(back_components[2:4], axis=0, keepdims=True)], axis=0)
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser, c_gammas, cbars_gamma = compute_mbars_cgammas_cbargammas(
                        mser_ibcm, coupling_eta_ibcm, basis)
sums_cbars_gamma = np.sum(cbars_gamma, axis=2)
sums_cbars_gamma2 = np.sum(cbars_gamma*cbars_gamma, axis=2)

# Analytical prediction, exact: need moments of nu. Easiest to compute numerically. 
# Odors are all iid so we can average over all odors
mean_conc = np.mean(conc_ser)
sigma2_conc = np.var(conc_ser)
thirdmom_conc = np.mean((conc_ser - mean_conc)**3)
moments_conc = [float(mean_conc), float(sigma2_conc), float(thirdmom_conc)]
print(moments_conc)

# Analytical prediction
res = fixedpoint_thirdmoment_exact(moments_conc, 1, n_components-1)
c_specif, c_nonspecif = res[:2]
cs_cn = res[:2]

# Count how many dot products are at each possible value. Use cbar = 1.0 as a split. 
split_val = 2.0
cbars_gamma_mean = np.mean(cbars_gamma[transient:], axis=0)
cgammas_bar_counts = {"above": int(np.sum(cbars_gamma_mean.flatten() > split_val)), 
                      "below": int(np.sum(cbars_gamma_mean.flatten() <= split_val))}
print(cgammas_bar_counts)

specif_gammas = np.argmax(np.mean(cbars_gamma[transient:], axis=0), axis=1)
print(specif_gammas)

# Analytical W
analytical_w = ibcm_fixedpoint_w_thirdmoment(inhib_rates, moments_conc, 
                                            basis, cs_cn, specif_gammas)

In [ ]:
fig, ax = plt.subplots()
#ax.plot(tser_ibcm[:300], nuser_ibcm[:300, :, 1])
neurons_cmap = sns.color_palette("Greys", n_colors=n_i_ibcm)
for i in range(n_i_ibcm):
    ax.plot(tser_ibcm/1000, thetaser_ibcm[:, i], lw=0.5, color=neurons_cmap[i])
ax.set(xlabel="Time (x1000 steps)", ylabel=r"$\bar{\Theta} = \bar{c}^2$ moving average")
plt.show()
plt.close()

In [ ]:
fig , ax, _ = plot_cbars_gamma_series(tser_ibcm, cbars_gamma, 
                        skp=10, transient=320000 // skp)
# Compare to exact analytical fixed point solution
#ax.set_xlim([200, 360])
ax.axhline(c_specif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{specific}}$")
ax.axhline(c_nonspecif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{non}}$")
fig.tight_layout()
leg = ax.legend(loc="upper left", bbox_to_anchor=(1., 1.))

#fig.savefig("figures/powerlaw/cbargammas_series_turbulent_background_example.pdf", 
#            transparent=True, bbox_inches="tight", bbox_extra_artists=(leg,))
plt.show()
plt.close()

In [ ]:
# Correlation between nu's and c's, see if some neurons are specific to odors
# Each neuron turns out to correlate its response to  one concentration
# that means it is specific to that odor. 
cbarser_norm_centered = cbarser_ibcm - np.mean(cbarser_ibcm[transient:], axis=0)
conc_ser_centered = (nuser_ibcm[:, :, 1] 
                     - np.mean(nuser_ibcm[transient:, :, 1], axis=0))
correl_c_nu = np.mean(cbarser_norm_centered[transient:, :, None] 
                      * conc_ser_centered[transient:, None, :], axis=0)

fig, ax = plt.subplots()
img = ax.imshow(correl_c_nu.T)
ax.set(ylabel=r"Component $\gamma$", xlabel=r"Neuron $i$")
fig.colorbar(img, label=r"$\langle (\bar{c}^i - \langle \bar{c}^i \rangle)"
             r"(\nu_{\gamma} - \langle \nu_{\gamma} \rangle) \rangle$", 
            location="top")
fig.tight_layout()
#fig.savefig("figures/powerlaw/specificities_turbulent_background_example.pdf", 
#           transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Check if each component has at least one neuron
for comp in range(n_components):
    print("Number of neurons specific to component {}: {}".format(
            comp, np.sum(np.mean(cbars_gamma[-2000:, :, comp], axis=0) > split_val)))

In [ ]:
fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                tser_ibcm, bkvecser_ibcm, yser_ibcm, skp=1)

# Compute noise reduction factor, annotate
transient = 100000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
#fig.savefig("figures/powerlaw/pn_activity_norm_turbulent_background_example.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, axes, _ = plot_background_neurons_inhibition(tser_ibcm, bkvecser_ibcm, yser_ibcm, skp=1)
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_ibcm, wser_ibcm, skp=100)
fig.tight_layout()
#fig.savefig("figures/powerlaw/w_series_turbulent_background_example.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## BioPCA simulation
### BioPCA habituation simulation

In [ ]:
# BioPCA model parameters
n_i_pca = n_components  # Number of inhibitory neurons for BioPCA case

# Model rates
learnrate_pca = 1e-4  # Learning rate of M
# Choose Lambda diagonal matrix as advised in Minden et al., 2018
# but scale it up to counteract W regularization
lambda_range_pca = 0.5
lambda_max_pca = 8.0
# Learning rate of L, relative to learnrate. Adjusted to Lambda in the integration function
rel_lrate_pca = 2.0  #  / lambda_max_pca**2 
lambda_mat_diag = build_lambda_matrix(lambda_max_pca, lambda_range_pca, n_i_pca)

xavg_rate_pca = learnrate_pca
pca_options = {
    "activ_fct": activ_function, 
    "remove_lambda": False, 
    "remove_mean": True
}
biopca_rates = [learnrate_pca, rel_lrate_pca, lambda_max_pca, lambda_range_pca, xavg_rate_pca]


# Initial synaptic weights: small positive noise
# We selected a seed (out of 40+ tested) giving initial conditions leading to correct PCA
# The model has trouble converging on this background, we're giving as many chances as possible here. 
rgen_pca = np.random.default_rng(seed=0x8b6664612cfeda4a121436fcfbbca449)
init_synapses_pca = rgen_pca.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_i_pca)
init_mmat_pca = rgen_pca.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_dimensions)
init_lmat_pca = np.eye(n_i_pca, n_i_pca)  # Supposed to be near-identity, start as identity
ml_inits_pca = [init_mmat_pca, init_lmat_pca]

In [ ]:
# Run simulation
sim_results = integrate_inhib_biopca_network_skip(
                ml_inits_pca, update_fct, init_back_list, biopca_rates, 
                inhib_rates, back_params, duration, deltat, 
                seed=simul_seed, noisetype="uniform", skp=skp, **pca_options)
(tser_pca, 
 nuser_pca, 
 bkvecser_pca, 
 mser_pca, 
 lser_pca, 
 xser_pca, 
 cbarser_pca, 
 wser_pca, 
 yser_pca) = sim_results

### BioPCA simulation analysis

In [ ]:
res = analyze_pca_learning(bkvecser_pca, mser_pca, lser_pca, 
                           lambda_mat_diag, demean=pca_options["remove_mean"])
true_pca, learnt_pca, fser, off_diag_l_avg_abs, align_error_ser = res

In [ ]:
fig, axes = plot_pca_results(tser_pca/1000, true_pca, learnt_pca, align_error_ser, off_diag_l_avg_abs)
axes[-1].set_xlabel("Time (x1000 steps)")
fig.set_size_inches(fig.get_size_inches()[0], 3*2.5)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                tser_pca, bkvecser_pca, yser_pca, skp=10)

# Compute noise reduction factor, annotate
transient = 100000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
fig, axes, _ = plot_background_neurons_inhibition(tser_pca, bkvecser_pca, yser_pca, skp=10)
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_pca, wser_pca, skp=10)
plt.show()
plt.close()

### Average background subtraction simulation

In [ ]:
# Average subtraction model parameters
avg_options = {"activ_fct": activ_function}

# Initial synaptic weights: dummy
init_synapses_avg = np.zeros([1, n_dimensions])

In [ ]:
sim_results = integrate_inhib_average_sub_skip(
                init_synapses_avg, update_fct, init_back_list, 
                [], inhib_rates, back_params, duration, deltat,
                seed=simul_seed, noisetype="uniform", skp=skp, **avg_options
)
tser_avg, bkser_avg, bkvecser_avg, wser_avg, yser_avg = sim_results

### Ideal inhibition
The component parallel to the background is reduced to beta / (2*alpha + beta). 

In [ ]:
back_projector = find_projector(back_components.T)
ideal_factor = inhib_rates[1] / (2*inhib_rates[0] + inhib_rates[1])
yser_ideal = bkvecser_ibcm * ideal_factor

### Orthogonal model

In [ ]:
yser_orthogonal = np.zeros(bkvecser_ibcm.shape)

### Optimal model

In [ ]:
from modelfcts.ideal import compute_optimal_matrices

In [ ]:
# New odor concentrations
new_test_concs = np.asarray([0.5, 1.0])
avg_whiff_conc = np.mean(truncexp1_average(*back_params[4:6]))
print("Average whiff concentration: {:.4f}".format(avg_whiff_conc))
new_test_concs *= avg_whiff_conc
n_new_concs = len(new_test_concs)

In [ ]:
# Compute optimal W matrix for all new odors possible
dummy_rgen = np.random.default_rng(0x115959f92ef489f1fe6b034c2ebfcb37)
n_samp = int(1e4)
new_odors_from_distrib = generate_odorant([n_samp, n_dimensions], 
                                    dummy_rgen, lambda_in=0.1)
new_odors_from_distrib /= l2_norm(new_odors_from_distrib)[:, None]
optimal_ws = compute_optimal_matrices(back_components, 
                new_odors_from_distrib, moments_conc, new_test_concs)

yser_optimal = bkvecser_ibcm - bkvecser_ibcm.dot(optimal_ws[0].T)

## Alignment between IBCM, PCA projectors and the subspace projector

In [ ]:
from utils.metrics import subspace_align_error
from utils.random_matrices import random_orthogonal_mat

In [ ]:
ibcm_matrix = np.mean(np.einsum("...ik,...kj", wser_ibcm[transient:], mbarser[transient:]), axis=0)
biopca_matrix = np.mean(np.einsum("...ik,...kj", wser_pca[transient:], fser[transient:]), axis=0)
align_err_ibcm = subspace_align_error(ibcm_matrix, back_projector)
align_err_pca = subspace_align_error(biopca_matrix, back_projector)
align_err_optimal = subspace_align_error(optimal_ws[0], back_projector)
align_err_random_matrix = subspace_align_error(random_orthogonal_mat(back_projector.shape[0], rng=dummy_rgen), back_projector)
print("Alignment error of IBCM projector vs orthogonal projector:", align_err_ibcm)
print("Alignment error of BioPCA projector vs orthogonal projector:", align_err_pca)
print("For comparison, alignment error of optimal manifold learning matrix:", align_err_optimal)
print("And that of a random orthogonal matrix:", align_err_random_matrix)
print("This tells us that both the IBCM mand BioPCA projectors are well aligned with the background subspace")
print("The lesser alignment for BioPCA could come from the approximate L inverse?")

## Model comparison for background inhibition

## Model comparison for background tagging after habituation
We create a projection matrix, then compute the tag assigned to the background after inhibition by each habituation model, over time. Hopefully, only IBCM inhibits enough to see tags go to zero. 

In [ ]:
# Common parameters
n_kc = 1000
projection_arguments = {
    "kc_sparsity": 0.05,
    "adapt_kc": True,
    "n_pn_per_kc": 3,
    "project_thresh_fact": 0.1
}
proj_mat = create_sparse_proj_mat(n_kc, n_dimensions, rgen_meta)

yser_dict = {
    "ibcm": yser_ibcm, 
    "biopca": yser_pca, 
    "avgsub": yser_avg, 
    "none": bkvecser_ibcm, 
    "ideal": yser_ideal, 
    "optimal": yser_optimal,
    "orthogonal": yser_orthogonal
}

## Model comparison for new odor recognition

In [ ]:
def find_snap_index(dt, skip, times):
    """ Find nearest multiple of dt*skip to each time in times """
    return np.around(times / (dt*skip)).astype(int)

In [ ]:
# Generate new odors, select test times, etc.
# New odors tested
n_new = 100
new_odors = generate_odorant([n_new, n_dimensions], rgen_meta, lambda_in=0.1)
new_odors /= l2_norm(new_odors)[:, None]

# Test times
n_test_times = 10
start_test_t = duration - n_test_times * 2000.0
test_times = np.linspace(start_test_t, duration, n_test_times)
test_times -= deltat*skp
test_idx = find_snap_index(deltat, skp, test_times)

# New odor concentrations
new_test_concs = np.asarray([0.5, 1.0])
avg_whiff_conc = np.mean(truncexp1_average(*back_params[4:6]))
print("Average whiff concentration: {:.4f}".format(avg_whiff_conc))
new_test_concs *= avg_whiff_conc
n_new_concs = len(new_test_concs)

# Background samples, indexed [time, sample, n_orn]
n_back_samples = 10
conc_samples = sample_ss_mixed_concs_powerlaw(
                    *back_params[:8], size=n_test_times*(n_back_samples-1), rgen=rgen_meta
                )  # Shaped [sample, component]
back_samples = conc_samples.dot(back_components)
back_samples = back_samples.reshape([n_test_times, n_back_samples-1, -1])
back_samples = np.concatenate([bkvecser_ibcm[test_idx, None, :], back_samples], axis=1)

# Containers for s vectors of each model
mixture_yvecs = {a: np.zeros([n_new, n_test_times,  n_new_concs,  
                    n_back_samples, n_dimensions]) for a in yser_dict.keys()}
mixture_tags = {a: SparseNDArray((n_new, n_test_times, n_new_concs,
                    n_back_samples, n_kc), dtype=bool) for a in yser_dict.keys()}
new_odor_tags = sparse.lil_array((n_new, n_kc), dtype=bool)
jaccard_scores = {a: np.zeros([n_new, n_test_times, n_new_concs,  n_back_samples]) 
                  for a in yser_dict.keys()}
jaccard_scores_with_back = {a: np.zeros([n_new, n_test_times, n_new_concs,  n_back_samples, n_components]) 
                  for a in yser_dict.keys()}

In [ ]:
# Compute ideal reduction factor for each concentration
dummy_rgen = np.random.default_rng(0x6e3e2886c30163741daaaf7c8b8a00e6)
ideal_factors = [compute_ideal_factor(c, moments_conc[:2], [n_components, n_dimensions], 
                    generate_odorant, (dummy_rgen,)) for c in new_test_concs]

# Compute background odor tags
back_tags = []
for i in range(n_components):
    back_tags.append(project_neural_tag(back_components[i], 
            back_components[i], proj_mat, **projection_arguments
    ))

for i in range(n_new):
    # Compute neural tag of the new odor alone, without inhibition
    new_tag = project_neural_tag(
                    new_odors[i], new_odors[i],
                    proj_mat, **projection_arguments
                )
    new_odor_tags[i, list(new_tag)] = True
    # Parallel and orthogonal components
    x_new_par = find_parallel_component(new_odors[i], 
                        back_components, back_projector)
    x_new_ort = new_odors[i] - x_new_par
    # Now, loop over snapshots, mix the new odor with the back samples,
    # compute the PN response at each test concentration,
    # compute tags too, and save results
    for j in range(n_test_times):
        jj = test_idx[j]
        for k in range(n_new_concs):
            mixtures = (back_samples[j]
                + new_test_concs[k] * new_odors[i])
            # odors, mlx, wmat, 
            # Compute for each model
            mixture_yvecs["ibcm"][i, j, k] = ibcm_respond_new_odors(
                mixtures, mser_ibcm[jj], wser_ibcm[jj], 
                ibcm_rates, options=ibcm_options
            )
            mixture_yvecs["biopca"][i, j, k] = biopca_respond_new_odors(
                mixtures, [mser_pca[jj], lser_pca[jj], xser_pca[jj]], 
                wser_pca[jj], biopca_rates, options=pca_options
            )
            mixture_yvecs["avgsub"][i, j, k] = average_sub_respond_new_odors(
                mixtures, wser_avg[jj], options=avg_options
            )
            mixture_yvecs["none"][i, j, k] = mixtures
            mixture_yvecs["ideal"][i, j, k] = ideal_linear_inhibitor(
                x_new_par, x_new_ort, back_samples[j], new_test_concs[k], 
                ideal_factors[k], **avg_options
            )
            mixture_yvecs["orthogonal"][i, j, k] = x_new_ort
            mixture_yvecs["optimal"][i, j, k] = mixtures - mixtures.dot(optimal_ws[k].T)
            for l in range(n_back_samples):
                for mod in mixture_yvecs.keys():
                    mix_tag = project_neural_tag(
                        mixture_yvecs[mod][i, j, k, l], mixtures[l],
                        proj_mat, **projection_arguments
                    )
                    try:
                        mixture_tags[mod][i, j, k, l, list(mix_tag)] = True
                    except ValueError as e:
                        print(mix_tag)
                        print(mixture_yvecs[mod][i, j, k, l])
                        print(proj_mat.dot(mixture_yvecs[mod][i, j, k, l]))
                        raise e
                    jaccard_scores[mod][i, j, k, l] = jaccard(mix_tag, new_tag)
                    # For comparison, compute similarity to each background odor
                    for m in range(n_components):
                        jaccard_scores_with_back[mod][i, j, k, l, m] = jaccard(mix_tag, back_tags[m])

In [ ]:
ideal_factors

In [ ]:
# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "optimal", "ideal", "avgsub", "biopca", "ibcm"]
for m in models:  # Plot IBCM last
    all_jacs = jaccard_scores[m]
    for i in range(n_new_concs):
        hist_outline(
            axes[i], all_jacs[:, :, i, :].flatten(),
            bins="doane", density=True, label=model_nice_names.get(m, m),
            color=model_colors.get(m), alpha=1.0
        )
        axes[i].axvline(
            np.median(all_jacs[:, :, i, :]), ls="--",
            color=model_colors.get(m)
        )
# Labeling the graphs, etc.
for i in range(n_new_concs):
    ax = axes[i]
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel("Jaccard similarity (higher is better)")
    axes[i].set_ylabel("Probability density")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
#fig.savefig("figures/detection/compare_models_onerun_{}.pdf".format(activ_fct),
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()

In [ ]:
# Distance to new odor
# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "optimal", "ideal", "avgsub", "biopca", "ibcm"]
all_medians = []
for m in models:  # Plot IBCM last
    all_distances = (mixture_yvecs[m] 
         - new_test_concs[None, None, :, None, None]*new_odors[:, None, None, None, :])
    all_norms = l2_norm(all_distances.reshape(-1, n_dimensions))
    all_medians.append(np.median(all_norms))
    for i in range(n_new_concs):
        hist_outline(
            axes[i], all_norms,
            bins="doane", density=True, label=model_nice_names.get(m, m),
            color=model_colors.get(m), alpha=1.0
        )
        axes[i].axvline(
            all_medians[-1], ls="--",
            color=model_colors.get(m)
        )
# Labeling the graphs, etc.
for i in range(n_new_concs):
    axes[i].set_xlim([0.0, 2.0*max(all_medians)])
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel(r"Distance to new odor, $\|\vec{s} - \vec{x}_{\mathrm{new}}\|$")
    axes[i].set_ylabel("Probability density")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
#fig.savefig("figures/detection/compare_models_onerun_ynorm_{}.pdf".format(activ_fct),
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()

In [ ]:
jaccard_scores_with_back["none"].shape

In [ ]:
# Scatter plot of Jaccard similarity to new odor alone vs similarity to most similar background odor
jaccard_scores_with_back_max = {a:np.amax(jaccard_scores_with_back[a], axis=4) for a in jaccard_scores_with_back}

# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "optimal", "ideal", "avgsub", "biopca", "ibcm", "orthogonal"]
for m in models:  # Plot IBCM last
    all_jacs = np.median(jaccard_scores[m], axis=(1,))  # Median time point, all back samples there
    all_jacs_back = np.median(jaccard_scores_with_back_max[m], axis=(1,))
    for i in range(n_new_concs):
        axes[i].scatter(all_jacs_back[:, i].flatten(), all_jacs[:, i].flatten(), 
            s=9.0, color=model_colors.get(m), alpha=0.5, label=model_nice_names[m])
# Labeling the graphs, etc.
for i in range(n_new_concs):
    ax = axes[i]
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", color="grey", lw=1.0, zorder=0)
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel("Jaccard with most similar background odor")
    axes[i].set_ylabel("Jaccard with new odor")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()

#fig.savefig(os.path.join("figures", "detection", "compare_models_onerun_jaccards_with_back.pdf"), 
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()

# Save some results for further plotting

## Background process
We need a supplementary simulation, fully sampled. 

In [ ]:
# Make a fully-sampled background simulation for figure 1
sim_results = integrate_inhib_average_sub_skip(
                init_synapses_avg, update_fct, init_back_list, 
                [], inhib_rates, back_params, duration, deltat,
                seed=simul_seed, noisetype="uniform", skp=1, **avg_options
)
tser_example, nuser_example = sim_results[:2]

In [ ]:
from modelfcts.checktools import check_turbulent_background_stats

In [ ]:
back_stats = check_turbulent_background_stats(nuser_example, back_params[:-3])

In [ ]:
# Plot the statistics to make sure it's convenient and correct. 
fig, axes = plt.subplots(1, 3)
fig.set_size_inches(9, 4)
cmyk_blue = "#3E529F"
cmyk_red = "#DA3833"
cmyk_green = "#307F54"

# Concentrations
ax = axes[0]
ax.bar(x=back_stats["conc_bins"][:-1], align="edge", height=back_stats["conc_pdf"], 
       width=np.diff(back_stats["conc_bins"]), color=cmyk_blue, label="Samples")
ax.plot(0.0, back_stats["conc_prob_zero"], color=cmyk_blue, marker="o", ls="none", mec="k", ms=8)
ax.plot(np.linspace(back_stats["conc_bins"][0], back_stats["conc_bins"][-1], 201), 
        back_stats["conc_analytical_pdf"], color=cmyk_red, lw=3.,
        label=r"$p_c(c) \sim \frac{e^{-c/c_0}}{Ac}$")
ax.plot(0.0, back_stats["conc_analytical_prob_zero"], marker="*", color=cmyk_red, ls="none",
        mec=cmyk_red, ms=4.5)
ax.set(xlabel=r"$c$", ylabel=r"$p_c(c)$", yscale="log")

# Blanks
ax = axes[1]
ax.bar(x=back_stats["t_b_bins"][:-1], align="edge", height=back_stats["t_b_pdf"], 
        width=np.diff(back_stats["t_b_bins"]), color=cmyk_blue, label="Samples")
ax.plot(back_stats["t_b_axis"], back_stats["t_b_analytical_pdf"], color=cmyk_red, lw=3.,
        label=r"$p(t_d) \sim t_d^{-3/2}$")
ax.set(xlabel=r"$t_b$", ylabel=r"$p_t(t_b)$", yscale="log", xscale="log")
       

# Whiffs
ax = axes[2]
ax.bar(x=back_stats["t_w_bins"][:-1], align="edge", height=back_stats["t_w_pdf"], 
        width=np.diff(back_stats["t_w_bins"]), color=cmyk_blue, label="Samples")
ax.plot(back_stats["t_w_axis"], back_stats["t_w_analytical_pdf"], color=cmyk_red, lw=3.,
        label=r"$p(t_w) \sim t_w^{-3/2}$")
ax.set(xlabel=r"$t_w$", ylabel=r"$p_t(t_w)$", yscale="log", xscale="log")

fig.tight_layout()
plt.show()
plt.close()

In [ ]:
results_dir = "results/for_plots/"
if do_save_outputs:
    np.savez_compressed(results_dir + "sample_turbulent_background.npz", 
        nuser=nuser_example, 
        tser=tser_example, 
        **back_stats
    )

In [ ]:
# Save the PCA simulation and its analysis too
save_skp = 2  # Extra skip for saving?
results_dir = "results/for_plots/"
if do_save_outputs:
    np.savez_compressed(
        os.path.join(results_dir, "sample_turbulent_biopca_simulation.npz"), 
        true_pca_values=true_pca[0], 
        true_pca_vectors=true_pca[1],
        learnt_pca_values=learnt_pca[0][::save_skp],
        learnt_pca_vectors=learnt_pca[1][::save_skp],
        align_error_ser=align_error_ser[::save_skp]
    )